## FEniCSx implementation

### The complete program

A FEniCSx program for solving our test problem for the Poisson equation
in 2D with the given choices of $\Omega$, $u_D$, and $f$ may look as
follows:

In [ ]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=4,
    ny=4
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 1)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return 1 + x[0]**2 + 2 * x[1]**2

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = fem.Constant(msh, -6.)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

### Convergence with $P_1$-elements

Use the code above with the manufactured solution $u_e(x,y) = 1 + x^2 + 2y^2$ to complete the following table for the approximation with $P_1$-elements:

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $4$ | ${1 \over 4} \sqrt{2}$ | 0.0329 | 0.3244 |
| $8$ | ${1 \over 8} \sqrt{2}$ | 0.0082 | 0.1616 |
| $16$ | ${1 \over 16} \sqrt{2}$ | 0.0021 | 0.0807 |
| $32$ | ${1 \over 32} \sqrt{2}$ | 0.0005 | 0.0403 |
| $64$ | ${1 \over 64} \sqrt{2}$ | 0.0001 | 0.0201 |
| $128$ | ${1 \over 128} \sqrt{2}$ | 0.00003 | 0.0100 |

Q: What are the convergence rates that these results suggest for the $L^2$-error and the $H^1$-error, respectively?

A: 

The $L^2$-error convergence rates are approximately 2, with some variation:
[2.004,1.965,2.070,2.322,1.737]

The $H^1$-error convergence rates are approximately 1:
[1.005,1.001,1.001,1.004,1.007]

In [1]:
import numpy as np

# Example data
h_values = [1/4 * np.sqrt(2), 1/8 * np.sqrt(2), 1/16 * np.sqrt(2), 1/32 * np.sqrt(2), 1/64 * np.sqrt(2), 1/128 * np.sqrt(2)]  # Example mesh sizes
L2_errors = [0.0329, 0.0082, 0.0021, 0.0005, 0.0001, 0.00003]  # Example L2 errors
H1_errors = [0.3244, 0.1616, 0.0807, 0.0403, 0.0201, 0.0100]  # Example H1 errors

# Compute convergence rates
def compute_convergence_rates(h_values, errors):
    rates = []
    for i in range(len(h_values) - 1):
        rate = np.log(errors[i + 1] / errors[i]) / np.log(h_values[i + 1] / h_values[i])
        rates.append(rate)
    return rates

# Compute rates for L2 and H1 errors
L2_rates = compute_convergence_rates(h_values, L2_errors)
H1_rates = compute_convergence_rates(h_values, H1_errors)

# Display results
print("L2 convergence rates:", L2_rates)
print("H1 convergence rates:", H1_rates)

L2 convergence rates: [np.float64(2.0043917691171576), np.float64(1.9652345818393238), np.float64(2.070389327891398), np.float64(2.321928094887362), np.float64(1.7369655941662063)]
H1 convergence rates: [np.float64(1.0053466214654785), np.float64(1.0017866194740144), np.float64(1.0017888347498127), np.float64(1.003584337349039), np.float64(1.007195501404204)]



Now switch from $P_1$-elements to $P_2$-elements and check that the numerical solution is exact (up to rounding error).

In [8]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=4,
    ny=4
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 2)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return 1 + x[0]**2 + 2 * x[1]**2

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = fem.Constant(msh, -6.)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

L2-error: 3.4737992217892883e-15
H1-error: 2.846434189904547e-14


| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $4$ | ${1 \over 4} \sqrt{2}$ | 3.4737992217892883e-15 | 2.846434189904547e-14 |
| $8$ | ${1 \over 8} \sqrt{2}$ | 1.4191143607449725e-14 | 8.443388496678458e-14 |
| $16$ | ${1 \over 16} \sqrt{2}$ | 5.550399557490119e-14 | 2.8458358983460907e-13 |
| $32$ | ${1 \over 32} \sqrt{2}$ | 2.188103854767406e-13 | 1.0622872132932086e-12 |
| $64$ | ${1 \over 64} \sqrt{2}$ | 8.935391921764368e-13 | 4.256001080125365e-12 |
| $128$ | ${1 \over 128} \sqrt{2}$ | 3.616430161582769e-12 | 1.7128858711713004e-11 |

### Convergence with higher-order elements

Create a new test problem by choosing a quintic polynomial as manufactured solution:

$$u_e(x,y) = x^5 + y^5$$

Q: What source term $f$ and what boundary values $u_D$ make this the exact solution of the Poisson-Dirichlet problem

\begin{align*}
-\Delta u (x,y) &= f(x,y) && (x,y) \in \Omega\\
u (x,y) &= u_D(x,y) && (x,y) \in \partial\Omega
\end{align*}

A: 

\begin{align*}
f(x,y) &= -20 x^3 - 20 y^3\\
u_D(x,y) &= x^5 + y^5
\end{align*}

Implement this new manufactured solution in the code above. Then measure the $L^2$-errors and $H^1$-errors when using $P_1$, $P_2$ or $P_3$ elements:

In [25]:
# P1 elements
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=8,
    ny=8
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 1)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return x[0]**5 + x[1]**5

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
x = SpatialCoordinate(msh)
f = -20 * x[0]**3 - 20 * x[1]**3
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

L2-error: 0.017665605600345344
H1-error: 0.38280356899717516


#### $P_1$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ | ${1 \over 8} \sqrt{2}$  | $0.0176$ |$0.3828$|
| $16$ | ${1 \over 16} \sqrt{2}$  | $0.0044$ |$0.1925$|
| $32$ | ${1 \over 32} \sqrt{2}$ | $0.0011$ |$0.0964$|
| $64$ | ${1 \over 64} \sqrt{2}$ | $0.0002$ |$0.0482$|

Estimated convergence rates:
- ... convergence for the $L^2$ errors = The $L^2$-error convergence rates are 2.0, with only one variation:
[2.0, 2.0, 2.459]

- ... convergence for the $H^1$ errors = The $H^1$-error convergence rates are all approximately 1.0:
[0.9917, 0.9978, 1.0]

In [24]:
# P1 convergence rates
import numpy as np

# Example data
h_values = [1/8 * np.sqrt(2), 1/16 * np.sqrt(2), 1/32 * np.sqrt(2), 1/64 * np.sqrt(2)]  # Example mesh sizes
L2_errors = [0.0176, 0.0044, 0.0011, 0.0002]  # Example L2 errors
H1_errors = [0.3828, 0.1925, 0.0964, 0.0482]  # Example H1 errors

# Compute convergence rates
def compute_convergence_rates(h_values, errors):
    rates = []
    for i in range(len(h_values) - 1):
        rate = np.log(errors[i + 1] / errors[i]) / np.log(h_values[i + 1] / h_values[i])
        rates.append(rate)
    return rates

# Compute rates for L2 and H1 errors
L2_rates = compute_convergence_rates(h_values, L2_errors)
H1_rates = compute_convergence_rates(h_values, H1_errors)

# Display results
print("L2 convergence rates:", L2_rates)
print("H1 convergence rates:", H1_rates)

L2 convergence rates: [np.float64(2.0), np.float64(2.0), np.float64(2.4594316186372973)]
H1 convergence rates: [np.float64(0.9917323840163995), np.float64(0.9977533942396645), np.float64(1.0)]


In [31]:
# P2 elements
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=8,
    ny=8
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 2)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return x[0]**5 + x[1]**5

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
x = SpatialCoordinate(msh)
f = -20 * x[0]**3 - 20 * x[1]**3
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

L2-error: 0.0004238997669224463
H1-error: 0.02197597281797851


#### $P_2$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ | ${1 \over 8} \sqrt{2}$  | $0.0004$ |$0.0219$|
| $16$ | ${1 \over 16} \sqrt{2}$ | $0.00005$ |$0.0055$|
| $32$ | ${1 \over 32} \sqrt{2}$ | $0.000007$ |$0.0014$|
| $64$ | ${1 \over 64} \sqrt{2}$ | $0.0000008$ |$0.0003$|

Estimated convergence rates:
- ... convergence for the $L^2$ errors = The $L^2$-error convergence rates are within the theoretical 3.0:
[3.0, 2.836, 3.1293]

- ... convergence for the $H^1$ errors = The $H^1$-error convergence rates are all approximately equal to the theoretical 2.0:
[1.9934, 1.9740, 2.2223]

In [30]:
# P2 convergence rates
import numpy as np

# Example data
h_values = [1/8 * np.sqrt(2), 1/16 * np.sqrt(2), 1/32 * np.sqrt(2), 1/64 * np.sqrt(2)]  # Example mesh sizes
L2_errors = [0.0004, 0.00005, 0.000007, 0.0000008]  # Example L2 errors
H1_errors = [0.0219, 0.0055, 0.0014, 0.0003]  # Example H1 errors

# Compute convergence rates
def compute_convergence_rates(h_values, errors):
    rates = []
    for i in range(len(h_values) - 1):
        rate = np.log(errors[i + 1] / errors[i]) / np.log(h_values[i + 1] / h_values[i])
        rates.append(rate)
    return rates

# Compute rates for L2 and H1 errors
L2_rates = compute_convergence_rates(h_values, L2_errors)
H1_rates = compute_convergence_rates(h_values, H1_errors)

# Display results
print("L2 convergence rates:", L2_rates)
print("H1 convergence rates:", H1_rates)

L2 convergence rates: [np.float64(3.0), np.float64(2.836501267717121), np.float64(3.129283016944967)]
H1 convergence rates: [np.float64(1.993427346076514), np.float64(1.9740047914670553), np.float64(2.222392421336448)]


In [36]:
# P3 elements
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=8,
    ny=8
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 3)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return x[0]**5 + x[1]**5

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
x = SpatialCoordinate(msh)
f = -20 * x[0]**3 - 20 * x[1]**3
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

L2-error: 7.437238166253042e-06
H1-error: 0.0005680401142416761


#### $P_3$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ | ${1 \over 8} \sqrt{2}$ | $0.000007$ |$0.0006$|
| $16$ | ${1 \over 16} \sqrt{2}$ |$0.0000005$ |$0.00007$|
| $32$ | ${1 \over 32} \sqrt{2}$ |$0.00000003$ |$0.000009$|
| $64$ | ${1 \over 64} \sqrt{2}$ |$0.000000002$ |$0.000001$|

Estimated convergence rates:
- ... convergence for the $L^2$ errors = The $L^2$-error convergence rates are all approximately within the theoretical 4.0:
[3.8073, 4.0589, 3.9069]

- ... convergence for the $H^1$ errors = The $H^1$-error convergence rates are all approximately within the theoretical 3.0:
[3.0995, 2.9593, 3.1699]

In [37]:
# P3 convergence rates
import numpy as np

# Example data
h_values = [1/8 * np.sqrt(2), 1/16 * np.sqrt(2), 1/32 * np.sqrt(2), 1/64 * np.sqrt(2)]  # Example mesh sizes
L2_errors = [0.000007, 0.0000005, 0.00000003, 0.000000002]  # Example L2 errors
H1_errors = [0.0006, 0.00007, 0.000009, 0.000001]  # Example H1 errors

# Compute convergence rates
def compute_convergence_rates(h_values, errors):
    rates = []
    for i in range(len(h_values) - 1):
        rate = np.log(errors[i + 1] / errors[i]) / np.log(h_values[i + 1] / h_values[i])
        rates.append(rate)
    return rates

# Compute rates for L2 and H1 errors
L2_rates = compute_convergence_rates(h_values, L2_errors)
H1_rates = compute_convergence_rates(h_values, H1_errors)

# Display results
print("L2 convergence rates:", L2_rates)
print("H1 convergence rates:", H1_rates)

L2 convergence rates: [np.float64(3.8073549220576046), np.float64(4.058893689053568), np.float64(3.9068905956085187)]
H1 convergence rates: [np.float64(3.0995356735509145), np.float64(2.9593580155026538), np.float64(3.1699250014423126)]
